In [1]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM

In [2]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data

data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')

In [3]:
def setup_data(data):
    """simplify setting up data correctly"""
    #calculate hausmann insturments
    mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
    plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
    hausman_instr = plan_dum

    #set up x and y
    y = data[['Inside Good Share','Outside Good Share']]
    x =  data[['Network Score','Satisfaction Score','PPO','Premium']]
    return x,y,hausman_instr

x,y,z =  setup_data(data)

# Calculating $\delta$, $\theta_1$, $\xi_{jt}$

Where $\theta_1$ = $(\alpha,\beta)$

In [4]:
#set up useful global variables 
NMKTS = data['Market_ID'].nunique()
NPLANS = data['Plan_ID'].nunique()
NOBS = data['Plan_ID'].count()
NSIM = 20

theta2 = np.array([3,2,1]) # initialize theta2 for testing purposes
delta = np.ones(NOBS)*(-2)

#print global variables
print NMKTS,NPLANS,NOBS

600 16 3300


In [5]:
#set up random draws v
mkt_ids = np.array(data['Market_ID'])
v = [0]*NOBS

for mkt in range(1,601):
    v_i = np.random.normal(size=(3,NSIM))
    for i in range(NOBS):
        if mkt_ids[i] == mkt:
            v[i] = v_i
        
v = np.array(v).transpose()
print v.shape

(20, 3, 3300)


In [6]:
def cal_s(data, v, delta, theta2):
    """Calculate market share
    Calculates individual choice probability first, then take sum"""
    
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO']]).transpose()
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1,3300))
    delta  = np.tile( delta  ,(NSIM,1))
    
    #add to calcualte market shares
    sim_exp = pd.DataFrame( np.exp(delta + (theta2*v*x).sum(axis=1)).transpose() ) 
    
    #sum up between markets
    sim_exp['mkt_id'] = data['Market_ID']
    sum_exp = sim_exp.groupby('mkt_id').sum() 
    sum_exp = pd.merge(data.copy()[['Market_ID']], sum_exp, 
                       right_index=True, left_on = 'Market_ID')
    
    #format so I can broadcast
    sim_exp = np.array(sim_exp).transpose()[:-1]
    sum_exp = np.array(sum_exp).transpose()[1:] + 1
    
    shares = (1./NSIM)*(sim_exp/sum_exp).sum(axis=0)
    return shares


s = cal_s(data, v, delta, theta2)

print s.mean(), s.min(), s.max(), s.shape

0.08594089477368333 0.030724838843884536 0.2681433769789094 (3300,)


In [7]:
def cal_delta(data, v, theta2, error = 1e-3, maxiter = 500):
    """Calculate mean utility via contraction mapping"""

    niter = 0
    
    #initialize loop parameters
    delta = np.zeros(NOBS)
    s = cal_s(data, v, delta, theta2)
    diff = np.log(data['Inside Good Share']) - np.log(s)
    
    
    while ((abs(diff).max() > 1e-6) #this is easier to converge
           and (abs(diff).mean() > error) 
           and niter < maxiter):
        
        s = cal_s(data, v, delta, theta2)
        diff = np.log(data['Inside Good Share']) - np.log(s)

        if np.isnan(diff).sum():
            raise Exception('nan in diffs')
            
        delta += diff
        niter += 1

    return delta


delta = cal_delta(data, v, theta2)
print delta.shape
print delta.mean()

(3300,)
2.08637190192


In [8]:
#initialize theta1 i.e. alpha and beta
theta1 = np.array([3,2,1,-2])

def cal_xi(data, delta, theta1):
    """Calculate xi with F.O.C"""
    x,y,z =  setup_data(data)
    xi = delta - np.matmul(np.array(x),theta1)
    return  xi


xi = cal_xi(data, delta,  theta1)
print xi.max(), xi.min(), xi.mean()

5.002521648309545 -1.0403798003429399 2.22526702241


# Calculating  $\theta_2$


Use GMM over $\theta_2$, the non-linear parameters. 

In [18]:
def cal_theta1(data, delta, T):
    """  calculate theta 1 using FOCs (X1'Z T Z'X )^-1 X1'Z T Z' delta """
    #set up variables
    x,y,z =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    
    #build up to main equation
    XtZ = X.transpose().dot(Z)
    ZtX = Z.transpose().dot(X)
        
    first_exp = np.linalg.inv( XtZ.dot(T).dot(ZtX))
    second_exp = XtZ.dot(T).dot(Z.transpose()).dot(delta)
    theta1 = first_exp.dot(second_exp)
    return theta1



T =  np.linalg.inv( z.transpose().dot(z) )
print cal_theta1(data, delta, T)



[ 3.5210287   1.53549582  0.62325933 -1.04963271]


In [33]:
def gmm_objective(theta2_init, data, v,  T):
    """Z'ww'Z of theta2"""
    #set up variables
    x,y,z =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    
    #do calculations
    delta = cal_delta(data, v, theta2_init)
    theta1 = cal_theta1(data, delta, T)
    xi = cal_xi(data, delta, theta1)
    return xi.dot(Z).dot(T).dot(Z.transpose()).dot(xi)
    

def calc_theta2(data, v, T, theta2_init):
    """calculate theta2 using scipy"""
    theta2 = minimize(gmm_objective, theta2_init, args=(data, v,  T), method='BFGS',
                      options={'maxiter':100, 'disp': True})
    return theta2.x



#run some tests
theta2_init = np.array([3,2,1])
theta2_test = calc_theta2( data, v, T, theta2_init)

print theta2_test

Optimization terminated successfully.
         Current function value: 0.281188
         Iterations: 11
         Function evaluations: 60
         Gradient evaluations: 12
[0.16019059 0.08523177 0.90731871]


In [36]:
def calc_theta(data, v, theta2_init):
    
    #initialize theta
    x,y,z =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    theta2 = theta2_init 
    
    #on first step, use consistent approximation of T
    T =  np.linalg.inv( z.transpose().dot(z) )
    for i in range(2):  
       
        #on second use T using estimated xi
        if i==1:
            xi = cal_xi(data, delta, theta1)
            xi =np.array([xi]).transpose()
            T =  np.linalg.inv( Z.transpose().dot(xi).dot(xi.transpose()).dot(Z) )
        
        theta2 = calc_theta2(data, v, T, theta2)
        delta = cal_delta(data, v, theta2)
        theta1 = cal_theta1(data, delta, T)
        
    return theta1, theta2


theta = calc_theta(data, v, theta2_init)

print theta

Optimization terminated successfully.
         Current function value: 0.281188
         Iterations: 11
         Function evaluations: 60
         Gradient evaluations: 12
         Current function value: 0.103191
         Iterations: 1
         Function evaluations: 372
         Gradient evaluations: 72
(array([ 3.68008423,  2.06858434,  0.86110783, -2.13258096]), array([0.16019059, 0.08523177, 0.9073187 ]))
